## Color scales in Bokeh

There isn't an obvious way to plot a color bar when using bokeh.

Here are a few ideas on how to look at that data:

  1. Make a color bar in a separate plot, then add them together using gridspec
  2. Use the hovertool to display the pixel values (hide some other glyphs under image)
  3. Use the hovertool to display the pixel values (tons of rectangles)

The right answer is anything that can display(save) quantitative information about the color scale in the same plot (PNG).

In [7]:
import numpy as np
from bokeh.models import Range1d, HoverTool, ColumnDataSource
from bokeh.models.mappers import LinearColorMapper
from bokeh.plotting import figure, show, output_notebook, gridplot
output_notebook()

BokehJS successfully loaded.

I'm going to use a Bokeh-tized version of the viridis color map for this. 

In [8]:
import matplotlib.pyplot as plt
import numpy as np
def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb[0:3]
def return_bokeh_colormap(name):
    cm = plt.cm.get_cmap(name)
    colormap = [rgb_to_hex(tuple((np.array(cm(x))*255).astype(np.int))) for x in range(0,cm.N)]
    return colormap

viridis = return_bokeh_colormap('viridis')

### Method 1

Color bar in separate figure

In [9]:
# pick colormap
cmap = viridis

# generate fake data
N = 25
imx = np.linspace(0,1,N)
imy = np.linspace(0,1,N)
IMX, IMY = np.meshgrid(imx, imy)
data = (np.cos(2*(IMX**2 + IMY**2)*np.pi) + np.sin(2*(IMX**2 + IMY**2)*np.pi))
dmin = data.min().min()
dmax = data.max().max()

# setup colorbar data
cbx = np.ones(len(cmap))*0.5
cby = np.linspace(data.min(), dmax, len(cmap))

color_bar_yrange = Range1d(dmin,dmax)
color_bar_xrange = Range1d(0.0,1.0)

# make colorbar
color_bar = figure(width=120, height=360, title=None, 
                   y_axis_location='right', y_range=color_bar_yrange, x_range=color_bar_xrange)
# color_bar.logo = None
color_bar.toolbar_location = None

color_bar.rect(x=cbx, y=cby, width=1, height=(dmax-dmin)/(len(cmap)-1), color=viridis,
    width_units='data', height_units='data')

color_bar.xgrid.grid_line_color = None
color_bar.ygrid.grid_line_color = None
color_bar.xaxis.visible = None

# show colorbar
# show(color_bar)

im = figure(width=420, height=360, title='datnum', title_text_font_size='10pt')
im.image(image=[data], x=[0], y=[0], dw=[1], dh=[1], color_mapper=LinearColorMapper(cmap))
im.set(x_range=Range1d(0, 1.0), y_range=Range1d(0, 1.0))

plot = gridplot([[im, color_bar]])

show(plot)

That's not bad. The tools are kind of a mess and you cannot save the image + color bar as a single PNG.

### Method 2

HoverTool using glyphs hidden behind the image

In [10]:
# # not yet working

# # pick colormap
# cmap = viridis

# # generate fake data
# imx = np.linspace(0,1,25)
# imy = np.linspace(0,1,25)
# IMX, IMY = np.meshgrid(imx, imy)
# data = (np.cos(2*(IMX**2 + IMY**2)*np.pi) + np.sin(2*(IMX**2 + IMY**2)*np.pi))
# dmin = data.min().min()
# dmax = data.max().max()

# TOOLS='pan,wheel_zoom,box_zoom,resize,reset,save,hover'
# im = figure(width=420, height=360, title='datnum', tools=TOOLS, title_text_font_size='10pt')
# im.logo=None

# im.image(image=[data], x=[0], y=[0], dw=[1], dh=[1], color_mapper=LinearColorMapper(cmap))
# im.set(x_range=Range1d(0, 1.0), y_range=Range1d(0, 1.0))

# show(im)

I'm abandoning this unless I prove it is somehow faster than Method 3.

### Method 3

HoverTool with toooooons of little rectangles

In [11]:
from collections import OrderedDict

def float_to_uint8(a):
    """ Convert np.float array to np.uint8 array. """
    dmin = np.amin(a[np.isfinite(a)])
    dmax = np.amax(a[np.isfinite(a)])
    
    idata = a.flatten()
    idata[np.isnan(idata)] = dmin
    idata = (idata-dmin)*255/(dmax-dmin)
    return idata.astype(np.uint8)

# pick colormap
cmap = viridis

# generate fake data
N = 25
imx = np.linspace(0,1,N)
imy = np.linspace(0,1,N)
IMX, IMY = np.meshgrid(imx, imy)
data = (np.cos(2*(IMX**2 + IMY**2)*np.pi) + np.sin(2*(IMX**2 + IMY**2)*np.pi))
dmin = np.amin(data)
dmax = np.amax(data)

TOOLS='pan,wheel_zoom,box_zoom,resize,reset,save,hover'
im = figure(width=420, height=360, title='datnum', tools=TOOLS, title_text_font_size='10pt')
im.logo=None

# create a column data source for the plots to share

cidx = float_to_uint8(data)
colors = [cmap[i] for i in cidx]
source = ColumnDataSource(data=dict(x=IMX.flatten(), y=IMY.flatten(), data=data.flatten(), color=colors, cidx=cidx))
im.rect('x','y', width=1.0/(N-1), height=1.0/(N-1), color='color', line_color = 'color', source=source)
im.set(x_range=Range1d(0, 1.0), y_range=Range1d(0, 1.0))
hover = im.select(dict(type=HoverTool))
hover.tooltips = OrderedDict([
    ('value', '@data'),
])

show(im)

I like this. It is pretty satisfying to play around with. However, I'm stuck with the same problem that I cannot save any information about the color scale in a PNG.